# Testing the Picoscope

See readme.

## Imports and Connecting

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import time

In [ ]:
import ctypes
from picosdk.ps4000 import ps4000 as ps
from picosdk.functions import adc2mV

chandle = ctypes.c_int16()

# Open PicoScope 4000 Series device
# Returns handle to chandle for use in future API functions
assert 0 == ps.ps4000OpenUnit(ctypes.byref(chandle))

## Channel parameters

In [ ]:
print( ps.PS4000_RANGE )
channel_range = ps.PS4000_RANGE['PS4000_5V']

for ch in 'AB':
    assert 0 == ps.ps4000SetChannel(chandle,
        ps.PS4000_CHANNEL[f'PS4000_CHANNEL_{ch}'],
        1, # enabled
        1, #DC coupling
        channel_range)

## Number of samples to acquire

In [ ]:
# Size of capture
sizeOfOneBuffer = int(np.round(1e6))
numBuffersToCapture = 30

totalSamples = sizeOfOneBuffer * numBuffersToCapture

# Create buffers ready for assigning pointers for data collection
bufferAMax = np.zeros(shape=sizeOfOneBuffer, dtype=np.int16)
bufferBMax = np.zeros(shape=sizeOfOneBuffer, dtype=np.int16)

memory_segment = 0

# Set data buffer location for data collection
# handle = chandle
# source = PS4000_CHANNEL_A  or B
# pointer to buffer max = ctypes.byref(bufferAMax)
# pointer to buffer min = ctypes.byref(bufferAMin)
# buffer length = maxSamples
# segment index = 0 ???
# ratio mode = PS4000_RATIO_MODE_NONE = 0
assert 0 == ps.ps4000SetDataBuffers(chandle,
    ps.PS4000_CHANNEL['PS4000_CHANNEL_A'],
    bufferAMax.ctypes.data_as(ctypes.POINTER(ctypes.c_int16)),
    None,
    sizeOfOneBuffer)

assert 0 == ps.ps4000SetDataBuffers(chandle,
    ps.PS4000_CHANNEL['PS4000_CHANNEL_B'],
    bufferBMax.ctypes.data_as(ctypes.POINTER(ctypes.c_int16)),
    None,
    sizeOfOneBuffer)

## Setup Streaming Mode

In [ ]:
# Begin streaming mode:
sampleInterval = ctypes.c_int32(150)
print(ps.PS4000_TIME_UNITS.keys())
sampleUnits = ps.PS4000_TIME_UNITS['PS4000_NS']
actualSampleInterval = sampleInterval.value
print(f"Capturing {totalSamples} samples at sample interval {actualSampleInterval} ns for a total duration of {actualSampleInterval*totalSamples/1e9} s")

# We are not triggering:
maxPreTriggerSamples = 0
autoStopOn = 1

# No downsampling:
downsampleRatio = 1

assert 0 == ps.ps4000RunStreaming(chandle,
    ctypes.byref(sampleInterval),
    sampleUnits,
    maxPreTriggerSamples,
    totalSamples,
    autoStopOn,
    downsampleRatio,
    sizeOfOneBuffer);


# We need a big buffer, not registered with the driver, to keep our complete capture in.
whole_acquire_A = np.zeros(shape=totalSamples, dtype=np.int16)
whole_acquire_B = np.zeros(shape=totalSamples, dtype=np.int16)

## Begin Streaming

In [ ]:
nextSample = 0
autoStopOuter = False
wasCalledBack = False


rng = np.arange(bufferAMax.size)
def streaming_callback(handle, noOfSamples, buff_start_idx, overflow, triggerAt, triggered, autoStop, param):
    global nextSample, autoStopOuter, wasCalledBack
    wasCalledBack = True
    idx = np.arange(buff_start_idx, buff_start_idx+noOfSamples) % bufferAMax.size
    whole_acquire_A[nextSample:nextSample + noOfSamples] = bufferAMax[idx]
    whole_acquire_B[nextSample:nextSample + noOfSamples] = bufferBMax[idx]
    nextSample += noOfSamples
    if autoStop:
        autoStopOuter = True


# Convert the python function into a C function pointer.
cFuncPtr = ps.StreamingReadyType(streaming_callback)

# Fetch data from the driver in a loop, copying it out of the registered buffers and into our complete one.
while nextSample < totalSamples and not autoStopOuter:
    ps.ps4000GetStreamingLatestValues(chandle, cFuncPtr, None)
    # if wasCalledBack:
    #     time.sleep(0.001)

print("Done grabbing values.")
# Stop the scope
ps.ps4000Stop(chandle)

# Disconnect the scope
ps.ps4000CloseUnit(chandle)

### Plot

In [ ]:
# Find maximum ADC count value
# pointer to value = ctypes.byref(maxADC)
maxADC = ctypes.c_int16(32767)

# Convert ADC counts data to mV
chA_mV = adc2mV(whole_acquire_A, channel_range, maxADC) #TPO edited this function, was doing the ADC conversion iteratively and returning a list
chB_mV = adc2mV(whole_acquire_B, channel_range, maxADC)

# Create time data
times = np.linspace(0, totalSamples * actualSampleInterval, totalSamples)/1e9

# Plot data from channel A and B
plt.figure()
plt.plot(times, chA_mV)
plt.plot(times, chB_mV)
plt.xlabel('Time (s)')
plt.ylabel('Voltage (mV)');

In [ ]:
print( 'dt', (times[1]-times[0]), 's' )
print( 'fs', 1/(times[1]-times[0])/1e6, 'MHz' );

In [ ]:
import h5py
import numpy as np

sys.exit()
with h5py.File('C:\\Users\\Regan Lab/Desktop\\\\\\test.hdf5','w') as f:
    grp = f.create_group('test')
    dset = grp.create_dataset('test', (100,))
    dset[:] = np.linspace(0,1,100)
